In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
import pandas as pd
import numpy as np

In [3]:
# df = pd.read_csv('/content/drive/MyDrive/hour_dataset.csv', encoding='EUC-KR')
df = pd.read_csv('./hour_dataset.csv', encoding='EUC-KR')

In [4]:
df.head()

,지점,일시,기온(°C),강수량(mm),풍속(m/s),풍향(16방위),습도(%),증기압(hPa),이슬점온도(°C),현지기압(hPa),...,운형(운형약어),최저운고(100m ),시정(10m),지면상태(지면상태코드),현상번호(국내식),지면온도(°C),5cm 지중온도(°C),10cm 지중온도(°C),20cm 지중온도(°C),30cm 지중온도(°C)
0,112,2023-01-01 00:00,3.0,NaN,4.6,230,62,4.7,-3.5,1020.8,...,NaN,NaN,872,NaN,40.0,-0.6,-0.1,-0.2,0.5,1.5
1,112,2023-01-01 01:00,3.2,NaN,5.1,250,64,4.9,-2.9,1020.5,...,NaN,15.0,793,NaN,40.0,-0.6,-0.1,-0.1,0.5,1.5
2,112,2023-01-01 02:00,2.8,NaN,4.1,270,70,5.2,-2.1,1020.4,...,NaN,12.0,618,NaN,40.0,-1.0,-0.1,-0.1,0.5,1.6
3,112,2023-01-01 03:00,2.3,NaN,2.8,320,71,5.1,-2.4,1020.6,...,Sc,10.0,545,NaN,40.0,-0.8,0.0,-0.1,0.5,1.5
4,112,2023-01-01 04:00,1.3,NaN,3.3,340,73,4.9,-3.0,1020.7,...,Sc,10.0,588,NaN,40.0,-0.9,-0.1,-0.2,0.5,1.5


In [5]:
df.keys()

Index(['지점', '일시', '기온(°C)', '강수량(mm)', '풍속(m/s)', '풍향(16방위)', '습도(%)',
       '증기압(hPa)', '이슬점온도(°C)', '현지기압(hPa)', '해면기압(hPa)', '일조(hr)',
       '일사(MJ/m2)', '적설(cm)', '3시간신적설(cm)', '전운량(10분위)', '중하층운량(10분위)',
       '운형(운형약어)', '최저운고(100m )', '시정(10m)', '지면상태(지면상태코드)', '현상번호(국내식)',
       '지면온도(°C)', '5cm 지중온도(°C)', '10cm 지중온도(°C)', '20cm 지중온도(°C)',
       '30cm 지중온도(°C)'],
      dtype='object')

In [6]:
df.shape

(8760, 27)

필요한 것들
- 기온
- 강수량
- 풍속
- 습도
- 현지기압
- 전운량
> 나머지는 지우자

In [7]:
labelsToRemove = ['지점', '일시', '풍향(16방위)',
'증기압(hPa)', '이슬점온도(°C)', '해면기압(hPa)', '일조(hr)',
'일사(MJ/m2)', '적설(cm)', '3시간신적설(cm)', '중하층운량(10분위)',
'운형(운형약어)', '최저운고(100m )', '시정(10m)', '지면상태(지면상태코드)', '현상번호(국내식)',
'지면온도(°C)', '5cm 지중온도(°C)', '10cm 지중온도(°C)', '20cm 지중온도(°C)',
'30cm 지중온도(°C)']

In [8]:
for label in labelsToRemove:
	df = df.drop(label, axis=1)

In [9]:
df.head()

,기온(°C),강수량(mm),풍속(m/s),습도(%),현지기압(hPa),전운량(10분위)
0,3.0,NaN,4.6,62,1020.8,8
1,3.2,NaN,5.1,64,1020.5,7
2,2.8,NaN,4.1,70,1020.4,7
3,2.3,NaN,2.8,71,1020.6,8
4,1.3,NaN,3.3,73,1020.7,8


단위까지 붙어있으면 접근하기 귀찮으니 변경하자

In [10]:
df = df.rename(
	columns = {
		"기온(°C)" : "기온",
		"강수량(mm)" : "강수량",
		"풍속(m/s)" : "풍속",
		"습도(%)" : "습도",
		"현지기압(hPa)" : "기압",
		"전운량(10분위)" : "운량"
	}
)
df.head()

,기온,강수량,풍속,습도,기압,운량
0,3.0,NaN,4.6,62,1020.8,8
1,3.2,NaN,5.1,64,1020.5,7
2,2.8,NaN,4.1,70,1020.4,7
3,2.3,NaN,2.8,71,1020.6,8
4,1.3,NaN,3.3,73,1020.7,8


결측값(NaN) 값을 체크해보자

In [11]:
for key in df.keys():
	print(sum(df[key].isna()))

0
7792
0
0
0
0


다행히 강수량에만 결측값이 있다.
> 그러나 강수량이 NaN 값인 것은 그냥 비가 안 온 거지 계측을 못했다는 것이 아니므로 0으로 채운다

In [12]:
df = df.fillna(0)
df.head()

,기온,강수량,풍속,습도,기압,운량
0,3.0,0.0,4.6,62,1020.8,8
1,3.2,0.0,5.1,64,1020.5,7
2,2.8,0.0,4.1,70,1020.4,7
3,2.3,0.0,2.8,71,1020.6,8
4,1.3,0.0,3.3,73,1020.7,8


이제 적절히 데이터의 범위를 변경하기 위해 최소 최댓값을 체크한다

In [13]:
for key in df.keys():
	print(key, min(df[key]), max(df[key]))

기온 -16.7 34.6
강수량 0.0 30.4
풍속 0.0 11.3
습도 10 99
기압 984.1 1030.1
운량 0 10


- 기온 | -20 ~ 40 => 0 ~ 1.
- 강수량 | 0 => 0 / >0 => 1
- 풍속 | 0 ~ 12. => 0 ~ 1.
- 습도 | 0 ~ 100 => 0 ~ 1.
- 운량 | 0 ~ 3 => 0 / 4 ~ 7 => 1
- 기압 | 980 ~ 1050 => 0 ~ 1.

In [14]:
df['기온'] = (df['기온'] + 20) / 60
df['강수량'] = df['강수량'].apply(lambda x: 1 if x > 0 else 0)
df['풍속'] = df['풍속'] / 12
df['습도'] = df['습도'] / 100
df['기압'] = (df['기압'] - 980) / 70
df['운량'] = df['운량'].replace([0,1,2,3], 0).replace([4,5,6,7,8,9,10], 1)

In [15]:
df.head()

,기온,강수량,풍속,습도,기압,운량
0,0.383333,0,0.383333,0.62,0.582857,1
1,0.386667,0,0.425000,0.64,0.578571,1
2,0.380000,0,0.341667,0.70,0.577143,1
3,0.371667,0,0.233333,0.71,0.580000,1
4,0.355000,0,0.275000,0.73,0.581429,1


In [16]:
dataY = np.array(df['강수량'])
dataY

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [17]:
dataX = np.array(list(zip(df['기온'], df['운량'], df['풍속'], df['습도'], df['기압'])))
dataX

array([[0.38333333, 1.        , 0.38333333, 0.62      , 0.58285714],
       [0.38666667, 1.        , 0.425     , 0.64      , 0.57857143],
       [0.38      , 1.        , 0.34166667, 0.7       , 0.57714286],
       ...,
       [0.36166667, 0.        , 0.2       , 0.87      , 0.51857143],
       [0.35      , 0.        , 0.075     , 0.89      , 0.52714286],
       [0.345     , 0.        , 0.21666667, 0.9       , 0.53285714]])

In [18]:
dataX.shape, dataY.shape

((8760, 5), (8760,))

이제 학습 고고씽

In [19]:
from sklearn.model_selection import train_test_split

trainX, testX, trainY, testY = train_test_split(dataX, dataY, test_size=0.2)


c:\Users\th070\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\__init__.py:177: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [51]:
import tensorflow as tf

In [52]:
model = tf.keras.models.Sequential([
	tf.keras.layers.Dense(64, activation='relu', input_shape=(5,)),
	tf.keras.layers.Dense(128, activation='relu'),
	tf.keras.layers.Dense(256, activation='relu'),
	tf.keras.layers.Dense(512, activation='relu'),
	tf.keras.layers.Dense(1, activation='sigmoid'),
])
# model = tf.keras.Sequential([
#   tf.keras.layers.Dense(1, activation='sigmoid', input_shape=(5,))  # 입력 특성 5개, 출력 1개 (운량 예측)
# ])

c:\Users\th070\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [53]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [54]:
model.fit(trainX, trainY, epochs=100, validation_data=(testX, testY))

Epoch 1/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.9212 - loss: 0.2493 - val_accuracy: 0.9486 - val_loss: 0.1325
Epoch 2/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9438 - loss: 0.1440 - val_accuracy: 0.9481 - val_loss: 0.1219
Epoch 3/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9435 - loss: 0.1312 - val_accuracy: 0.9492 - val_loss: 0.1235
Epoch 4/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9407 - loss: 0.1394 - val_accuracy: 0.9452 - val_loss: 0.1222
Epoch 5/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9385 - loss: 0.1455 - val_accuracy: 0.9509 - val_loss: 0.1185
Epoch 6/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9398 - loss: 0.1384 - val_accuracy: 0.9475 - val_loss: 0.1182
Epoch 7/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9478 - loss: 0.1225 - val_accuracy: 0.9481 - val_loss: 0.1196
Epoch 8/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9458 - loss: 0.1248 - val_accu

KeyboardInterrupt: 

In [56]:
model.evaluate(testX, testY)

55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9587 - loss: 0.1229


[0.11701390147209167, 0.9543378949165344]

In [25]:
# import datetime
# now = datetime.datetime.now()
# model.save(f'/content/drive/MyDrive/sdgsmodels/weather_{now.strftime("%Y%m%d_%H%M%S")}.h5')
# model.save(f'E:\GithubProjects\WeatherPredictionSDGS\models\weathermodel_v1.h5')

RandomForest

In [57]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score

In [58]:
model = RandomForestRegressor(n_estimators=200, random_state=42)
model.fit(trainX, trainY)

# 예측
prediction = model.predict(testX)
print(f"{sum((prediction > 0.5) == (testY == 1)) / len(prediction) * 100}%")

95.662100456621%


In [49]:
for n_esti in range(1, 30):
	model = RandomForestRegressor(n_estimators=n_esti * 10, random_state=42)
	model.fit(trainX, trainY)

	# 예측
	prediction = model.predict(testX)
	print(f"{n_esti * 10} {sum((prediction > 0.5) == (testY == 1)) / len(prediction) * 100}%")

10 95.14840182648402%
20 95.662100456621%
30 95.60502283105022%
40 95.37671232876713%
50 95.77625570776256%
60 95.83333333333334%
70 95.662100456621%
80 95.71917808219177%
90 95.54794520547945%
100 95.54794520547945%
110 95.662100456621%
120 95.60502283105022%
130 95.60502283105022%
140 95.71917808219177%
150 95.662100456621%
160 95.662100456621%
170 95.71917808219177%
180 95.71917808219177%
190 95.71917808219177%
200 95.662100456621%
210 95.60502283105022%
220 95.60502283105022%
230 95.54794520547945%
240 95.60502283105022%
250 95.71917808219177%
260 95.71917808219177%
270 95.71917808219177%
280 95.662100456621%
290 95.77625570776256%


60일때 가장 정확도가 높다

In [59]:
model = RandomForestRegressor(n_estimators=60, random_state=42)
model.fit(trainX, trainY)

RandomForestRegressor(n_estimators=60, random_state=42)